# **Emoji Classification Project - Computer Vision**

### Team Members : `Daniil NOTKIN`, `Yuhan SU` & `Yassine ERRAJI`

## *Importations*

In [ ]:
from warnings import filterwarnings
filterwarnings("ignore")

import os
import numpy as np
import pandas as pd 
import cv2
from pathlib import Path
from PIL import Image
from skimage import io, color
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models, optimizers

## *Functions*

In [ ]:
def load_single_image(path):
    """
    This will be the function you use to preprocess your image.
    
    """
    img = Image.open(path).convert("RGBA")
    img = np.array(img)

    # RGBA -> RGB
    if img.ndim == 3 and img.shape[2] == 4:
        img = color.rgba2rgb(img) 
    # grayscale -> RGB
    elif img.ndim == 2:
        img = np.stack([img] * 3, axis=-1)

    # unint8 conversion
    if img.dtype != np.uint8:
        img = (img * 255).astype(np.uint8)

    # here you could add additional preprocessing

    return img.astype("float32") / 255.0


# ============================================================
# Improved imageLoader 
# - shuffles every epoch
# - resizes images
# - optional data augmentation
# - optional sample weights (to replace class_weight)
# ============================================================


def imageLoader(
    files,
    labels,
    batch_size,
    target_size=(72, 72),
    augment=False,
    class_weight=None,   # NEW: dict like {class_index: weight}
    seed=42              # NEW: for reproducibility if you want it
):
    """
    Python generator yielding:
      - (X, y) if class_weight is None
      - (X, y, sample_weight) if class_weight is provided

    Notes:
    - labels must be a dict: {img_id (string): label_name (string)}
    - label_to_index must already exist in the notebook scope
    """

    rng = np.random.default_rng(seed)

    files = np.array(files)
    L = len(files)

    while True:
        # Shuffle at the beginning of each epoch
        indices = rng.permutation(L)
        files_shuffled = files[indices]

        batch_start = 0
        while batch_start < L:
            batch_files = files_shuffled[batch_start:batch_start + batch_size]

            X_batch = np.empty((len(batch_files), target_size[1], target_size[0], 3), dtype=np.float32)
            y_batch = np.empty((len(batch_files),), dtype=np.int32)

            for i, f in enumerate(batch_files):
                img_id = Path(f).stem  # DO NOT CHANGE (constraint)

                # Load image (float32, [0,1])
                img = load_single_image(f)

                # Resize to fixed shape
                img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)

                # Data augmentation (train only)
                if augment:
                    # Horizontal flip
                    if rng.random() < 0.5:
                        img = cv2.flip(img, 1)

                    # Small rotation
                    if rng.random() < 0.3:
                        angle = rng.uniform(-15, 15)
                        M = cv2.getRotationMatrix2D(
                            (target_size[0] // 2, target_size[1] // 2),
                            angle,
                            1.0
                        )
                        img = cv2.warpAffine(
                            img,
                            M,
                            (target_size[0], target_size[1]),
                            flags=cv2.INTER_LINEAR,
                            borderMode=cv2.BORDER_REFLECT_101
                        )

                # Store
                X_batch[i] = img
                y_batch[i] = label_to_index[labels[img_id]]

            if class_weight is None:
                yield X_batch, y_batch
            else:
                # Convert per-class weights into per-sample weights
                sample_weight = np.array([class_weight[int(c)] for c in y_batch], dtype=np.float32)
                yield X_batch, y_batch, sample_weight

            batch_start += batch_size

### Dataset

In [ ]:
PATH = Path("data")

In [ ]:
train_dir = PATH / "train"
train_files = sorted([str(p) for p in train_dir.iterdir() if p.is_file()])

In [ ]:
test_dir = PATH / "test"
test_files = sorted([str(p) for p in test_dir.iterdir() if p.is_file()])
test_ids = [Path(f).stem for f in test_files]

In [ ]:
y_train_df = pd.read_csv(PATH / "train_labels.csv")

y_train_dct = dict(zip(y_train_df["Id"], y_train_df["Label"]))

In [ ]:
gen = imageLoader(
    files=train_files,
    labels=y_train_dct,
    batch_size=32,
)

## Training

In [ ]:
# ============================================================
# Train / Validation split
# ============================================================

# Split train files into train / validation
train_files_split, val_files_split = train_test_split(
    train_files,
    test_size=0.2,        # 80% train / 20% validation
    random_state=42,
    shuffle=True
)

print(f"Train samples: {len(train_files_split)}")
print(f"Validation samples: {len(val_files_split)}")

In [ ]:
unique_labels = y_train_df["Label"].unique().tolist()
unique_labels

### Label encoding (strings → integers)

Keras requires numeric labels.

In [ ]:
# ============================================================
# Convert label dict keys to STRING to match Path(f).stem
# ============================================================

labels_str = {
    f"{int(k):05d}": v
    for k, v in y_train_dct.items()
}

In [ ]:
# ============================================================
# Label encoding (string → integer)
# ============================================================

# Sorted for reproducibility
unique_labels = sorted(y_train_df["Label"].unique())

label_to_index = {label: i for i, label in enumerate(unique_labels)}
index_to_label = {i: label for label, i in label_to_index.items()}

num_classes = len(unique_labels)

print("Label to index mapping:")
print(label_to_index)

In [ ]:
# ============================================================
# Compute class weights to handle imbalance
# ============================================================

y_encoded = [label_to_index[label] for label in y_train_df["Label"]]

class_weights_array = compute_class_weight(
    class_weight="balanced",
    classes=np.arange(num_classes),
    y=y_encoded
)

class_weight = dict(enumerate(class_weights_array))

print("Class weights:")
print(class_weight)

In [ ]:
# Encode string labels to integers
label_to_index = {label: i for i, label in enumerate(unique_labels)}
index_to_label = {i: label for label, i in label_to_index.items()}

num_classes = len(unique_labels)
print(label_to_index)

### Define a compact, efficient CNN

This model:

•   Trains fast

•   Fits in memory

•   Should be enough for this task

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models, optimizers

def build_model(input_shape=(72, 72, 3), num_classes=7):
    """
    MobileNetV2-based classifier.
    Phase 1: backbone frozen
    Phase 2: partial fine-tuning
    """

    # Pretrained backbone
    base_model = MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights="imagenet"
    )

    # Phase 1: freeze entire backbone
    base_model.trainable = False

    # Classification head
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=base_model.input, outputs=outputs)

    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

### **Instantiate the model**

In [ ]:
# ============================================================
# Instantiate model
# ============================================================

model = build_model(num_classes=num_classes)
model.summary()

### Train the model

We must define steps_per_epoch manually because we use a generator.

In [ ]:
# ============================================================
# Training generators (consistent with 72x72 model input)
# ============================================================

batch_size = 32
target_size = (72, 72)

train_gen = imageLoader(
    files=train_files_split,
    labels=labels_str,          # keys must match Path(f).stem
    batch_size=batch_size,
    target_size=target_size,
    augment=True,
    class_weight=class_weight,  # IMPORTANT: generator will output sample weights
    seed=42
)

val_gen = imageLoader(
    files=val_files_split,
    labels=labels_str,
    batch_size=batch_size,
    target_size=target_size,
    augment=False,
    class_weight=None,          # no weights needed for validation
    seed=123
)

# Use ceil division so you don't drop the tail of the dataset
steps_per_epoch = int(np.ceil(len(train_files_split) / batch_size))
validation_steps = int(np.ceil(len(val_files_split) / batch_size))

print("steps_per_epoch:", steps_per_epoch)
print("validation_steps:", validation_steps)

In [ ]:
# ============================================================
# Training - Phase 1 (frozen backbone)
# ============================================================

history_phase1 = model.fit(
    train_gen,
    validation_data=val_gen,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    epochs=15,
    verbose=1
)

In [ ]:
# ============================================================
# Fine-tuning - Phase 2 
# ============================================================

import tensorflow as tf

# Number of layers to fine-tune from the end
FINE_TUNE_LAYERS = 30

# Freeze all layers first
for layer in model.layers:
    layer.trainable = False

# Unfreeze last N layers (except BatchNorm)
for layer in model.layers[-FINE_TUNE_LAYERS:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

# Recompile with lower learning rate (critical)
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history_phase2 = model.fit(
    train_gen,
    validation_data=val_gen,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    epochs=10,
    verbose=1
)

### **We now replace the dummy prediction function**

In [ ]:
def predict_with_my_solid_model(test_files):
    """
    Predict labels for the test set using the trained CNN.
    """
    predictions = []

    for f in test_files:
        img = load_single_image(f)
        img = np.expand_dims(img, axis=0)  # (1, H, W, C)

        probs = model.predict(img, verbose=0)
        pred_idx = np.argmax(probs, axis=1)[0]
        predictions.append(index_to_label[pred_idx])

    return predictions

### **Generate submission**

In [ ]:
y_test_pred = predict_with_my_solid_model(test_files)

test_ids_sr = pd.Series(test_ids, name="Id")
y_test_pred_sr = pd.Series(y_test_pred, name="Label")

submission_df = pd.concat([test_ids_sr, y_test_pred_sr], axis=1)
submission_df.head()

In [ ]:
submission_df.to_csv("submission.csv", index=False)